In [1]:
import sys
import os
import pandas
import numpy
import psycopg2

In [2]:
from dotenv import load_dotenv
load_dotenv()

DBNAME = os.getenv("dbname")
USERNAME = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")

# Kaggle dataset

[Link to Kaggle dataset, based on WordNet](https://www.kaggle.com/datasets/duketemon/hypernyms-wordnet)

In [3]:
df = pandas.read_csv("data/hypernyms.csv")

In [4]:
df

,lemma,part_of_speech,hypernyms
0,0,noun,digit
1,1,noun,digit
2,10,noun,large integer
3,100,noun,large integer
4,1000,noun,large integer
...,...,...,...
120375,zip by,verb,travel by
120376,zip up,verb,fasten
120377,zonk out,verb,change state|fall asleep
120378,zoom along,verb,travel rapidly


In [5]:
nouns_df = df[df["part_of_speech"] == "noun"][["lemma", "hypernyms"]]

In [6]:
nouns_df = nouns_df.dropna()
nouns_df = nouns_df.rename(columns={"lemma": "word", "hypernyms": "categories"})
nouns_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104504 entries, 0 to 119483
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   word        104504 non-null  object
 1   categories  104504 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB


In [7]:
# assert no duplicates
assert len(nouns_df) == len(nouns_df['word'].unique())
print("Passed")

Passed


## Some thoughts

- A lot of noisy data - not the best
- building up might be more efficient than building down: the most hypernyms a word has is 33
- What if there's a cycle? This dataset does not look good because it might have synonyms. We may have to use other datasets
    - Band-aid solution would be to remove the cycles
 
Some better resources:
- [EVALuation](https://paperswithcode.com/dataset/evalution)
- [EVALuation Github](https://github.com/esantus/EVALution/tree/master/EVALution_1.0)
- [Linked Hypernyms Dataset](https://ner.vse.cz/datasets/linkedhypernyms/)

In [8]:
nouns_df

,word,categories
0,0,digit
1,1,digit
2,10,large integer
3,100,large integer
4,1000,large integer
...,...,...
117570,obsess,medicine
118039,possess,police
118833,saw logs,log
119084,siss,female sibling


### Some data analysis

In [9]:
nouns_df[nouns_df["categories"].str.contains(';')].head(50)

,word,categories
57,aboriginal,australian;ethnic group|person
112,acting,activity;performing arts
126,adagio,musical composition;passage|dancing
127,adamant,carbon;transparent gem
246,allegretto,tempo|musical composition;passage
247,allegro,tempo|musical composition;passage
277,amethyst,quartz;transparent gem
298,andante,tempo|musical composition;passage
430,arsenical,drug;pesticide
443,asian,inhabitant;person of color


In [10]:
nouns_df[nouns_df["word"] == "melon"]

,word,categories
67179,melon,edible fruit|gourd


## Using PostgreSQL

In [11]:
conn = psycopg2.connect(
    dbname=DBNAME,
    user=USERNAME,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

cursor = conn.cursor()

In [12]:
print(conn.get_dsn_parameters())

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'linguini', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [13]:
def initCategoriesTable():
    queries = [
        "DROP TABLE IF EXISTS categories;",
        '''
        CREATE TABLE IF NOT EXISTS categories(
            word TEXT NOT NULL,
            category TEXT NOT NULL,
            PRIMARY KEY(word, category)
        );
        ''',
        '''
        CREATE INDEX IF NOT EXISTS word_idx ON categories(word);
        ''',
        '''
        CREATE INDEX IF NOT EXISTS category_idx ON categories(category);
        '''
    ]

    try:
        for query in queries:
            cursor.execute(query)

        conn.commit()
        print("Successfully initialized database.")
    except:
        conn.rollback()
        print("Categories Table initialization failed, rolling back")
        


initCategoriesTable()

Successfully initialized database.


In [14]:
# loading the data

def load_data():
    try:
        for idx, row in nouns_df.iterrows():
            word = row["word"]
            groups = row["categories"].split("|")
            categories = set()
            for group in groups:
                values = group.split(";")
                for value in values:
                    categories.add(value)
                   
            for category in categories:
                insert_query = "INSERT INTO categories VALUES (%s,%s)"
                data = (word, category)
                cursor.execute(insert_query, data)
        conn.commit()
        print("Successfully loaded words data into database")
    except Exception as e:
        conn.rollback()
        print(f"Error: ({e})")

load_data()

Successfully loaded words data into database


In [15]:
cursor.execute("SELECT * FROM categories LIMIT 10")
print(cursor.fetchall())

[('0', 'digit'), ('1', 'digit'), ('10', 'large integer'), ('100', 'large integer'), ('1000', 'large integer'), ('11', 'large integer'), ('12', 'large integer'), ('120', 'large integer'), ('13', 'large integer'), ('14', 'large integer')]


## Wrangling

In [16]:
def make_query(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        conn.rollback()
        print(f"An error occurred: {e}")

In [17]:
get_table_size = "SELECT pg_size_pretty(pg_total_relation_size('categories'));"
result = make_query(get_table_size)
print(result)

[('24 MB',)]


In [58]:
query = "SELECT word, category FROM categories WHERE category = 'region'"
result = make_query(query)

print(len(result))
print(result)

93
[('bottom', 'region'), ('exterior', 'region'), ('inside', 'region'), ('interior', 'region'), ('outside', 'region'), ('side', 'region'), ('sign', 'region'), ('top', 'region'), ('tops', 'region'), ('unknown', 'region'), ('aerospace', 'region'), ('air', 'region'), ('airs', 'region'), ('antipodes', 'region'), ('area', 'region'), ('atmosphere', 'region'), ('backwater', 'region'), ('belt', 'region'), ('biogeographical region', 'region'), ('biosphere', 'region'), ('black hole', 'region'), ('breadbasket', 'region'), ('country', 'region'), ('county', 'region'), ('deep space', 'region'), ('demesne', 'region'), ('depth', 'region'), ('distance', 'region'), ('district', 'region'), ('domain', 'region'), ('dominion', 'region'), ('eden', 'region'), ('edgeworth-kuiper belt', 'region'), ('extremity', 'region'), ('field', 'region'), ('field of operations', 'region'), ('fields', 'region'), ('geographic area', 'region'), ('geographic region', 'region'), ('geographical area', 'region'), ('geographical re

## Turn into recursion

### A test with cycles and simple data

Over here I was ensuring my logic was correct

In [25]:
def initTestTable():
    queries = [
        "DROP TABLE IF EXISTS test;",
        '''
        CREATE TABLE IF NOT EXISTS test(
            word TEXT NOT NULL,
            category TEXT NOT NULL,
            PRIMARY KEY(word, category)
        );
        ''',
        '''
        CREATE INDEX IF NOT EXISTS word_idx ON test(word);
        ''',
        '''
        CREATE INDEX IF NOT EXISTS category_idx ON test(category);
        '''
    ]

    try:
        for query in queries:
            cursor.execute(query)

        conn.commit()
        print("Successfully initialized test table.")
    except:
        conn.rollback()
        print("Test Table initialization failed, rolling back")
        


initTestTable()

Successfully initialized test table.


In [26]:
def load_test_data():
    try:
        queries = [
            "INSERT INTO test VALUES ('a','b')",
            "INSERT INTO test VALUES ('b','c')",
            "INSERT INTO test VALUES ('c','a')",
            "INSERT INTO test VALUES ('d','e')",
            "INSERT INTO test VALUES ('e','d')",
            "INSERT INTO test VALUES ('f','d')",
            
        ]
        
        for query in queries:
            cursor.execute(query)
        conn.commit()
        print("Successfully loaded into test database")
    except Exception as e:
        conn.rollback()
        print(f"Error: ({e})")

load_test_data()

Successfully loaded into test database


In [27]:
search_query = "SELECT * FROM test"
make_query(search_query)

[('a', 'b'), ('b', 'c'), ('c', 'a'), ('d', 'e'), ('e', 'd'), ('f', 'd')]

In [28]:
recursion_query = """
    WITH RECURSIVE 
    hypernyms(word, category) AS (
        SELECT word, category FROM test
        UNION
        SELECT h.word, t.category
        FROM hypernyms h, test t
        WHERE h.category = t.word AND h.word != t.category
    )
SELECT * FROM hypernyms;
"""

make_query(recursion_query)

[('a', 'b'),
 ('b', 'c'),
 ('c', 'a'),
 ('d', 'e'),
 ('e', 'd'),
 ('f', 'd'),
 ('c', 'b'),
 ('a', 'c'),
 ('b', 'a'),
 ('f', 'e')]

### Now with real data

In [19]:
group_by_category = '''
    SELECT category, COUNT(*) as count
    FROM categories
    GROUP BY category
    HAVING COUNT(*) >= 65
    ORDER BY count DESC;
'''

len(make_query(group_by_category))

212

### Next steps:
- How do you deal with groups and hypernyms of hypernyms?
- Group handling
- Dropping the groups with less than 50 words?

# MVP: Validate if a word is in a category

* It appears that the categories database is too big. If we're to hold the entire database somewhere, it'll practically explod. It might just be faster to search linearly.
* At most this is O(n) time but it's very rare.

In [56]:
# Reduced height of recursion: can only move up 3 at least

def validate(word, category) -> bool:   
    validation_query = """
       WITH RECURSIVE 
        search(word, category, depth) AS(
            SELECT word, category, 0 FROM categories WHERE word = %s

            UNION 

            SELECT s.word, c.category, s.depth + 1
            FROM search s, categories c
            WHERE s.category = c.word AND s.word != c.category AND s.depth < 5
        )

        SELECT * 
        FROM search
        WHERE word = %s AND category = %s   
    """
    try:
        cursor.execute(validation_query, (word, word, category))
        result = cursor.fetchall()
        print(result)
        return len(result) > 0
    except Exception as e:
        conn.rollback()
        print(f"An error occurred: {e}")


result = validate("snail", "mollusk")
print(result)

[('snail', 'mollusk', 1)]
True


### We need to get all categories that "have enough words"

In [54]:
get_categories = """
    SELECT category, COUNT(*)
    FROM categories
    GROUP BY category
    HAVING COUNT(*) >= 65
    ORDER BY COUNT(*)
"""

# a recursive, difficult query
result = make_query(get_categories)
print(len(result))
print(result)

212
[('common fraction', 65), ('list', 65), ('indian', 65), ('food', 66), ('artifact', 66), ('cut', 66), ('protoctist genus', 66), ('information', 66), ('animal', 66), ('situation', 67), ('note', 67), ('poisonous plant', 67), ('flavorer', 67), ('ball', 67), ('company', 68), ('framework', 68), ('gathering', 69), ('compass point', 69), ('board', 69), ('gas', 69), ('cavity', 69), ('pain', 69), ('antibiotic', 69), ('employee', 70), ('program', 70), ('musician', 71), ('animal disease', 71), ('aquatic plant', 71), ('tract', 71), ('legume', 71), ('group action', 72), ('neural structure', 72), ('communication', 72), ('work', 72), ('organic compound', 72), ('dynasty', 73), ('mollusk family', 74), ('criminal', 74), ('mechanical device', 74), ('phylum', 74), ('feeling', 74), ('electrical device', 74), ('disease', 74), ('large indefinite quantity', 75), ('fern family', 75), ('linear unit', 75), ('house', 76), ('bag', 76), ('vessel', 76), ('asian', 76), ('reflex', 77), ('amphibian genus', 77), ('do

In [57]:
import random

categories = [x[0] for x in make_query(get_categories)]

assert len(categories) == len(set(categories))

random.shuffle(categories)

score = 0
num_categories = 0;

guess = ""

idx = 0

while guess != "STOP":
    category = categories[idx % len(categories)]
    print(f"Category: {category} Current score: {score}")
    
    num_categories += 1
    
    guess = input("Enter your guess: ")
    if guess == "STOP":
        break
    while(not validate(guess, category) and guess != "SKIP" and guess != "STOP"):
        guess = input("Incorrect guess, try again: ")
        
    if guess != "SKIP":
        score += len(guess)
    idx += 1

if score >= 1000:
    print(f"Game complete! Final number of categories: {num_categories}")
else:
    print("You exited the game")

Category: program Current score: 0


Enter your guess:  engineering


[('engineering', 'program', 1), ('engineering', 'program', 3), ('engineering', 'program', 4), ('engineering', 'program', 5)]
Category: common fraction Current score: 11


Enter your guess:  denominator


[]


...

[]


Incorrect guess, try again:  north america


[]


Incorrect guess, try again:  STOP


[]
You exited the game
